# CT Scanner Simulation: Proof of Functionality (Group 6)

## Introduction

In this report we report the functionality of the Python CT scanner simulation that has been developed over the course of the first 2 weeks of the project GG2.

The main capabilities of this simulation are listed below:

1. X-Ray Source Definition for X-Ray Generation (`photons.py` & `fake_source.py`)

2. Modelling of X-Ray Scattering and Attenuation by Different Materials (class `Material`)

3. Simulation of the X-ray Detector Array (`ct_detect.py`)

4. Phantom Definition (`ct_phantom.py`)

5. Production of Uncalibrated Sinograms (`ct_scan.py`)

6. Production of Calibrated Sinograms (`ct_calibrate.py`)

7. Implementation of the Inverse Radon Transform for Reconstruction from Sinograms (`back_project.py` & `ramp_filter.py`)

8. Complete Implementation of the Scanning and Reconstruction Process for an Arbitrary Phantom (`scan_and_reconstruct.py`)

## 1. X-Ray Source Definition for X-Ray Generation

![Figure 1]( source.JPG "Figure 1: Schematic of a typical X-ray source. Reproduced from the GG2 Project Handout. Author: Graham Treece")
<center> <i> Figure 1: Schematic of a typical X-ray source. Reproduced from the GG2 Project Handout. Author: Graham Treece </i></center>


The modules `photons.py` and `fake_source.py` provide the X-ray source generation capability of our simulation. Sources with different energy distributions can be selected from the predefined sources stored in the `source` class. The different sources vary in filter type, filter width and the potential difference between anode and cathode (see Fig.1) in kVp which limits the maximum photon energy in the resulting distribution. The predefined options are listed below:

1. 100kVp, 1mm Al
2. 100kVp, 2mm Al
3. 100kVp, 3mm Al
4. 100kVp, 4mm Al
5. 80kVp, 1mm Al
6. 80kVp, 2mm Al
7. 80kVp, 3mm Al
8. 80kVp, 4mm Al

The source distributions for these options are plotted in Fig. 2:

![Figure 2]( source_distribution.JPG "Figure 2: Source Distribution for Predefined Options. Reproduced from GG2 Project Handout. Author: Graham Treece")

<center> <i> Figure 2: Source Distribution for Predefined Options. Reproduced from GG2 Project Handout. Author: Graham Treece </i></center>

In `fake_source.py` we also provide the option of generating an arbitrary source distribution by allowing the user to define a filter material, filter width, and maximum photon energy (through anode/cathode potential difference). This module further provides the option of defining an ideal source with an energy distribution centred on the maximum photon energy specified by the user.

## 2. Modelling of X-Ray Scattering and Attenuation by Different Materials

The `Material` class contains as attributes the names of 18 different materials and their corresponding attenuation coefficients for 200 energy levels (0.001 MeV - 0.2 MeV). The materials for which data is listed are:

1. Air              &nbsp;   &nbsp; &nbsp;  &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;  10. Chromium 
2. Adipose           &nbsp;  &nbsp;  &nbsp; &nbsp; &nbsp; &nbsp;  11. Iron
3. Soft Tissue &nbsp;    &nbsp; &nbsp; &nbsp;  12. Carbon
4. Breast Tissue &nbsp;  &nbsp;   13. Nickel
5. Water &nbsp; &nbsp;  &nbsp;   &nbsp; &nbsp;  &nbsp; &nbsp; &nbsp; 14. Magnesium
6. Blood &nbsp; &nbsp;  &nbsp;   &nbsp; &nbsp;  &nbsp; &nbsp; &nbsp; &nbsp; 15. Aluminium
7. Bone &nbsp; &nbsp;  &nbsp;   &nbsp; &nbsp;  &nbsp; &nbsp; &nbsp; &nbsp; 16. Copper 
8. Titanium &nbsp; &nbsp;  &nbsp;   &nbsp; &nbsp;  &nbsp; 17. Co-Cr 
9. Cobalt &nbsp; &nbsp;  &nbsp;   &nbsp; &nbsp;  &nbsp; &nbsp; &nbsp;  18. Stainless Steel

Typical attenuation coefficient distributions for 10 materials over the aforementioned energy range are shown in Fig.3:

![Figure 3]( attenuation_coeff_distribution.JPG "Figure 3: Attenuation Coefficient vs. Photon Energy for 10 materials in the Material Class. Reproduced from the GG2 Project Handout. Author: Graham Treece")

<center> <i> Figure 3: Attenuation Coefficient vs. Photon Energy for 10 materials in the Material Class. Reproduced from the GG2 Project Handout. Author: Graham Treece </i> </center>

## 3. Simulation of the X-ray Detector Array

`ct_detect.py` simulates the X-ray detector array used to measure the total residual energy of source rays after passing through a phantom. It implements the rule defined in *Equation 1*:

$$I_{tot} = \sum_{E} I_{0,E}  e^{\sum_{m} \mu_{m,E} x_m}  \; \; (Equation \; 1) $$ 


## 4. Phantom Definition

`ct_phantom.py` defines a phantom (`type array`) of specified dimensions using different material ellipses (with arbitrary minor/major axes and the capability of rotating the ellipse) of positive and negative "mass" to create different shapes. The function ct_phantom can return one of 7 different types of phantom, which are listed below:

1. Simple circle for looking at calibration issues
2. Point attenuator for looking at resolution
3. Single large hip replacement
4. Bilateral hip replacement
5. Sphere with three satellites
6. Disc and other sphere
7. Pelvic fixation pins

Types 3-7 re-create a metal hip implant whose material for the metallic element you can select by choosing the desired value of the 'metal' parameter, default is Titanium.

An example of a type 3 phantom is shown in Fig.4: 

![Figure 4]( normal_phantom.png "Figure 4: Type 3 Phantom - Single Large Titanium Hip Replacement (scale shows material index)")

<center> <i> Figure 4: Type 3 Phantom - Single Large Titanium Hip Replacement (scale shows material index) </i> </center>


`atten_coeff_ct_phantom.py` performs a similar task but instead of storing the material index in the phantom array it stores the material attenuation coefficient at a user-defined energy level. This phantom is useful to compare the results of a complete simulation using an ideal source matching the user-defined energy level.

An example of a phantom defined using the material attenuation coefficients for 0.069 MeV can be seen in Fig.5:

![Figure 5]( atten_coeff_phantom.png "Figure 5: Type 3 Phantom - Single Large Titanium Hip Replacement (scale shows attenuation coefficient at 0.069 MeV)")

<center> <i> Figure 5: Type 3 Phantom - Single Large Titanium Hip Replacement (scale shows attenuation coefficient at 0.069 MeV) </i> </center>

## 5. Production of Uncalibrated Sinograms

`ct_scan.py` performs a complete scan of a previously-defined phantom (of dimensions specified by the `scale` parameter) for a user-defined number of angles. The `ct_scan` function returns a sinogram of residual energies as computed by `ct_detect.py`, an example of which can be seen in Fig. 6:

![Figure 6]( scan_256.png "Figure 6: Residual Energy Sinogram for a Type 3 Phantom (as described in Section 4)")

<center> <i> Figure 6: Residual Energy Sinogram for a Type 3 Phantom (as described in Section 4) </i> </center>

This sinogram of residual energies must be converted into a sinogram of attenuation coefficients to be able to later reconstruct the original phantom. This process is performed by the `ct_calibrate.py` module.

## 6. Production of Calibrated Sinograms

`ct_calibrate.py` implements the approximation in Equation 2 to compute the total attenuation coefficient ($\mu_{tot}$) as seen by the X-ray after passing through a section of a phantom. 

$$\mu_{tot} = \sum_{m} \mu_{m,E} x_m \approx \sum_{m} \mu_{m} x_m = -\log_e{\frac{I_{tot}}{\sum_{E} I_{0,E} } }$$

The function defined therein returns a sinogram of attenuation coefficients, an example of which is shown in Fig. 7:

![Figure 7]( sinogram.png "Figure 7: Attenuation Coefficient Sinogram for a Type 3 Phantom (as described in Section 4)")

<center> <i> Figure 7: Attenuation Coefficient Sinogram for a Type 3 Phantom (as described in Section 4) </i> </center>


## 7. Implementation of the Inverse Radon Transform for Reconstruction from Sinograms


The inverse *Radon transform* is used to convert the sinogram of attenuation coefficients returned by the `ct_calibrate.py` module into a scalar field of attenuation coefficients $\mu(x,y)$ which will describe the phantom as in `atten_coeff_ct_phantom.py`. The analogy for this transform is that we are performing a filtered back-projection (see Fig. 8) of the sinogram over all angles used to define it onto an array of the same dimensions as the original phantom.



The implementation of this transform requires the definition and use of the Ram-Lak filter that is convolved with the sinogram. In the frequency domain, this reduces to a multiplication between the fourier transform of the Ram-Lak filter (see Fig. 9) and the fourier transform of the sinogram. The inverse fourier transform of this multiplication integrated over all the angles used to compute the sinogram results in the reconstruction of the original phantom.

In [5]:
from IPython.display import Video
Video("ideal_source_reconstruction_video.mp4")

In [6]:
Video("angle_differences_video.mp4")